In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Sample Submission.csv
/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Test.csv
/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Train.csv


In [2]:
train = pd.read_csv("/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Train.csv")
test = pd.read_csv("/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Test.csv")
samp = pd.read_csv("/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Sample Submission.csv")

In [3]:
train.head()

,Text_ID,Product_Description,Product_Type,Sentiment
0,3057,The Web DesignerÛªs Guide to iOS (and Android...,9,2
1,6254,RT @mention Line for iPad 2 is longer today th...,9,2
2,8212,Crazy that Apple is opening a temporary store ...,9,2
3,4422,The lesson from Google One Pass: In this digit...,9,2
4,5526,RT @mention At the panel: &quot;Your mom has a...,9,2


In [4]:
train['Product_Type'].value_counts()

9    4070
6     665
2     465
7     327
3     300
5     213
8     194
1      59
0      52
4      19
Name: Product_Type, dtype: int64

In [5]:
train['Sentiment'].value_counts()

2    3765
3    2089
1     399
0     111
Name: Sentiment, dtype: int64

In [6]:
train.groupby(['Product_Type'])['Sentiment'].value_counts()

Product_Type  Sentiment
0             3              47
              1               4
              2               1
1             3              53
              1               5
              2               1
2             3             379
              1              69
              2              15
              0               2
3             3             240
              1              49
              2              10
              0               1
4             3              19
5             3             171
              1              36
              2               6
6             3             563
              1              84
              2              16
              0               2
7             3             276
              1              43
              2               8
8             3             122
              1              65
              2               6
              0               1
9             2            3702
              3 

In [7]:
train.groupby(['Sentiment'])['Product_Type'].value_counts()

Sentiment  Product_Type
0          9                105
           2                  2
           6                  2
           3                  1
           8                  1
1          6                 84
           2                 69
           8                 65
           3                 49
           9                 44
           7                 43
           5                 36
           1                  5
           0                  4
2          9               3702
           6                 16
           2                 15
           3                 10
           7                  8
           5                  6
           8                  6
           0                  1
           1                  1
3          6                563
           2                379
           7                276
           3                240
           9                219
           5                171
           8                122
           1    

In [8]:
test['Product_Type'].value_counts()

9    1731
6     281
2     196
7     143
3     130
8     103
5      80
0      26
1      22
4      16
Name: Product_Type, dtype: int64

In [9]:
!pip install simpletransformers
!pip uninstall -y tokenizers
!pip install transformers

     |████████████████████████████████| 208 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 3.8 MB/s  eta 0:00:01
     |████████████████████████████████| 7.2 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 884 kB 54.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 48.1 MB/s eta 0:00:01


     |████████████████████████████████| 120 kB 61.3 MB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7423 sha256=bcf7ad5bbc0caa44e784add6bc1cbbbfde92a010e60e5511a248037de8656a35
  Stored in directory: /root/.cache/pip/wheels/dc/cc/62/a3b81f92d35a80e39eb9b2a9d8b31abac54c02b21b2d466edc
Successfully built seqeval


  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.45.0
    Uninstalling tqdm-4.45.0:
      Successfully uninstalled tqdm-4.45.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.1.1
    Uninstalling ipykernel-5.1.1:
      Successfully uninstalled ipykernel-5.1.1
  Attempting uninstall: cachetools
    Found existing installation: cachetools 3.1.1
    Uninstalling cachetools-3.1.1:
      Successfully uninstalled cachetools-3.1.1
  Attempting uninstall: transformers
    Found existing installation: transformers 2.11.0
    Uninstalling transformers-2.11.0:
      Successfully uninstalled transformers-2.11.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 3.1.0 requires tokenizers==0.8.1.r

In [10]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [11]:
import os
import gc
import re
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss

In [12]:
train.drop('Text_ID', axis=1, inplace=True)
test.drop('Text_ID', axis=1, inplace=True)

In [13]:
train[train.duplicated()]

,Product_Description,Product_Type,Sentiment
1523,"RT @mention ÷¼ Happy Woman's Day! Make love, ...",9,2
3868,RT @mention RT @mention Google to Launch Major...,9,2
4512,RT @mention Google to Launch Major New Social ...,9,2
4777,"#SXSW is just starting, #CTIA is around the co...",0,3
4991,"Oh. My. God. The #SXSW app for iPad is pure, u...",7,3
5436,RT @mention Marissa Mayer: Google Will Connect...,9,2
5480,Counting down the days to #sxsw plus strong Ca...,2,3
5745,RT @mention ÷¼ GO BEYOND BORDERS! ÷_ {link} ...,9,2
5858,RT @mention Google to Launch Major New Social ...,9,2


In [14]:
# print(train.shape)
# train = train.drop_duplicates()
# print(train.shape)

In [15]:
test[test.duplicated()]

,Product_Description,Product_Type
1436,RT @mention Google to Launch Major New Social ...,9
2457,RT @mention Marissa Mayer: Google Will Connect...,3


In [16]:
test[test['Product_Description'].isin(train['Product_Description'].values.tolist())]

,Product_Description,Product_Type
103,"Before It Even Begins, Apple Wins #SXSW {link}",2
266,Need to buy an iPad2 while I'm in Austin at #s...,6
303,RT @mention Marissa Mayer: Google Will Connect...,9
317,RT @mention Google to Launch Major New Social ...,9
586,Marissa Mayer: Google Will Connect the Digital...,9
876,Google to Launch Major New Social Network Call...,9
906,RT @mention Google to Launch Major New Social ...,9
1025,I just noticed DST is coming this weekend. How...,8
1094,RT @mention Marissa Mayer: Google Will Connect...,3
1395,Really enjoying the changes in Gowalla 3.0 for...,1


In [17]:
train = train.rename(columns={'Product_Description' : 'text', 'Sentiment': 'labels'})
test = test.rename(columns={'Product_Description' : 'text'})

In [18]:
train['text'] = train['text'].astype(str) + ' ' + train['Product_Type'].astype(str)
test['text'] = test['text'].astype(str) + ' ' + test['Product_Type'].astype(str)

In [19]:
train.drop('Product_Type', axis=1, inplace=True)
test.drop('Product_Type', axis=1, inplace=True)

In [20]:
print(train['text'].apply(lambda x: len(x.split())).describe())

count    6364.000000
mean       18.730830
std         4.965762
min         3.000000
25%        15.000000
50%        19.000000
75%        22.000000
max        34.000000
Name: text, dtype: float64


In [21]:
print(test['text'].apply(lambda x: len(x.split())).describe())

count    2728.000000
mean       18.846041
std         4.949445
min         5.000000
25%        15.000000
50%        19.000000
75%        23.000000
max        32.000000
Name: text, dtype: float64


In [22]:
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [23]:
model_args = {'train_batch_size': 128, 
              'reprocess_input_data': True,
              'overwrite_output_dir': True,
              'fp16': False,
              'do_lower_case': False,
              'num_train_epochs': 7,
              'max_seq_length': 34,
              'regression': False,
              'manual_seed': 1994,
              "learning_rate": 1e-5,
              #'weight_decay': 0.01,
              "save_eval_checkpoints": False,
              "save_model_every_epoch": False,
              'no_cache':True,
              "silent": False,
              "use_early_stopping": True,
              #"early_stopping_delta": 0.015,
              "early_stopping_metric": "mcc",
              "early_stopping_metric_minimize": False,
              "early_stopping_patience": 3,
              "evaluate_during_training_steps": 1000
              }

In [24]:
def get_model():
    model = ClassificationModel('roberta', 'roberta-base', use_cuda=True, num_labels=4, args=model_args)                            
    return model

In [25]:
err=[]
y_pred_tot=[]
i=1

fold=StratifiedKFold(n_splits=5, shuffle=True, random_state=1994)

for train_index, test_index in fold.split(train, train['labels']):
    train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
    model = get_model()
    gc.collect()

    model.train_model(train1_trn)
    score, raw_outputs_val, wrong_preds = model.eval_model(train1_val) 
    raw_outputs_val = softmax(raw_outputs_val, axis=1) 
    print('Log_Loss:', log_loss(train1_val['labels'], raw_outputs_val))
    err.append(log_loss(train1_val['labels'], raw_outputs_val))
    predictions, raw_outputs_test = model.predict(test['text'])
    raw_outputs_test = softmax(raw_outputs_test, axis=1) 
    y_pred_tot.append(raw_outputs_test)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.5512822086777237


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.574208609082349


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.6119374637956565


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.6194366897424889


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.5776664138472868


In [26]:
np.mean(err, 0)

0.586906277029101

In [27]:
y_pred = np.mean(y_pred_tot, 0)

In [28]:
samp.head()

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [49]:
samp[['0','1','2','3']] = y_pred
samp.head()

,0,1,2,3
0,0.006008,0.006244,0.711419,0.276329
1,0.016405,0.023132,0.928492,0.031971
2,0.014886,0.065645,0.354355,0.565115
3,0.005009,0.011859,0.067244,0.915888
4,0.005622,0.003685,0.931481,0.059212


In [50]:
train = pd.read_csv("/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Train.csv")
test = pd.read_csv("/kaggle/input/machine-hack-product-sentiment-classification/Participants_Data/Test.csv")

In [51]:
train.columns

Index(['Text_ID', 'Product_Description', 'Product_Type', 'Sentiment'], dtype='object')

In [52]:
test_ids = []
force_senti = []
for index,row in test.iterrows():
    temp = train[train['Product_Description'] == row['Product_Description']]
    temp = temp[temp['Product_Type'] == row['Product_Type']]
    if temp.shape[0] > 0:
        force_senti.append(list(set(temp['Sentiment'].tolist()))[0])
        test_ids.append(index)

In [53]:
dic = {0:[1,0,0,0],1:[0,1,0,0],2:[0,0,1,0],3:[0,0,0,1]}
for x,y in zip(test_ids,force_senti):
    #print(s1.iloc[x])
    target = dic[y]
    samp.iloc[x] = target
    #print(s1.iloc[x])

In [54]:
samp.head(3)

,0,1,2,3
0,0.006008,0.006244,0.711419,0.276329
1,0.016405,0.023132,0.928492,0.031971
2,0.014886,0.065645,0.354355,0.565115


In [55]:
samp.to_csv('Sub_v0.3.csv', index=False)